<a href="https://colab.research.google.com/github/elichen/aoc2018/blob/main/Day_24_Immune_System_Simulator_20XX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [116]:
data = """Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4""".split('\n')

In [117]:
data = """Immune System:
2317 units each with 2435 hit points (weak to bludgeoning, cold; immune to fire) with an attack that does 10 cold damage at initiative 2
967 units each with 5447 hit points (immune to cold; weak to slashing, radiation) with an attack that does 50 fire damage at initiative 3
851 units each with 11516 hit points with an attack that does 106 cold damage at initiative 7
4857 units each with 2273 hit points with an attack that does 4 radiation damage at initiative 17
1929 units each with 2540 hit points (immune to fire) with an attack that does 12 radiation damage at initiative 4
4673 units each with 3705 hit points with an attack that does 5 fire damage at initiative 16
2312 units each with 6698 hit points (immune to radiation) with an attack that does 26 slashing damage at initiative 15
3526 units each with 3316 hit points (weak to cold) with an attack that does 9 fire damage at initiative 20
3207 units each with 2502 hit points (weak to cold, fire; immune to slashing) with an attack that does 6 radiation damage at initiative 11
5213 units each with 9656 hit points (immune to radiation) with an attack that does 17 fire damage at initiative 6

Infection:
8026 units each with 11443 hit points (immune to bludgeoning, slashing, fire) with an attack that does 2 cold damage at initiative 14
4465 units each with 36617 hit points (weak to radiation) with an attack that does 16 cold damage at initiative 1
378 units each with 18782 hit points (weak to radiation; immune to fire) with an attack that does 98 slashing damage at initiative 10
1092 units each with 17737 hit points (immune to bludgeoning, fire; weak to slashing) with an attack that does 25 slashing damage at initiative 18
270 units each with 19361 hit points (weak to bludgeoning, slashing) with an attack that does 104 fire damage at initiative 9
2875 units each with 30650 hit points (weak to fire) with an attack that does 16 slashing damage at initiative 5
1024 units each with 43191 hit points (weak to bludgeoning, cold) with an attack that does 76 cold damage at initiative 12
892 units each with 10010 hit points (immune to radiation; weak to slashing) with an attack that does 15 radiation damage at initiative 19
2708 units each with 40667 hit points (immune to cold; weak to fire) with an attack that does 21 cold damage at initiative 13
780 units each with 44223 hit points (immune to bludgeoning, radiation, slashing) with an attack that does 104 cold damage at initiative 8""".split('\n')

In [118]:
import re

def parse_input(input_data):
    lines = input_data

    # Initialize data structures for both armies
    immune_system = []
    infection = []
    current_army = None

    # Regular expression to parse the group details
    group_regex = re.compile(
        r'(\d+) units each with (\d+) hit points (\([^)]*\) )?with an attack that does (\d+) (\w+) damage at initiative (\d+)'
    )

    for line in lines:
        if line.startswith('Immune System'):
            current_army = immune_system
        elif line.startswith('Infection'):
            current_army = infection
        elif line:
            match = group_regex.match(line)
            if match:
                units, hit_points, properties, attack_damage, attack_type, initiative = match.groups()
                group = {
                    'units': int(units),
                    'hit_points': int(hit_points),
                    'attack_damage': int(attack_damage),
                    'attack_type': attack_type,
                    'initiative': int(initiative),
                    'weaknesses': [],
                    'immunities': []
                }

                if properties:
                    # Process weaknesses and immunities
                    properties = properties.strip('() ')
                    for prop in properties.split('; '):
                        if prop.startswith('weak to'):
                            group['weaknesses'] = prop[len('weak to '):].split(', ')
                        elif prop.startswith('immune to'):
                            group['immunities'] = prop[len('immune to '):].split(', ')

                current_army.append(group)

    return immune_system, infection

In [119]:
def simulate_battle(immune_system, infection):
    def calculate_effective_power(group):
        return group['units'] * group['attack_damage']

    def calculate_damage(attacking, defending):
        if attacking['attack_type'] in defending['immunities']:
            return 0
        damage = calculate_effective_power(attacking)
        if attacking['attack_type'] in defending['weaknesses']:
            damage *= 2
        return damage

    while immune_system and infection:
        # print("--")
        # print("immune_system:", immune_system)
        # print("infection",infection)

        # Target selection
        all_groups = sorted([(g, 'immune') for g in immune_system] + [(g, 'infection') for g in infection],
                            key=lambda x: (calculate_effective_power(x[0]), x[0]['initiative']), reverse=True)
        targets = {}
        for group, group_type in all_groups:
            potential_targets = infection if group_type == 'immune' else immune_system
            potential_targets = sorted(potential_targets,
                                       key=lambda g: (calculate_damage(group, g), calculate_effective_power(g), g['initiative']),
                                       reverse=True)
            for target in potential_targets:
                if target not in targets.values() and calculate_damage(group, target) > 0:
                    targets[id(group)] = target
                    break

        # Attacking phase
        for group, group_type in sorted(all_groups, key=lambda x: x[0]['initiative'], reverse=True):
            if group['units'] <= 0:
                continue
            target = targets.get(id(group))
            if target:
                damage = calculate_damage(group, target)
                units_killed = min(target['units'], damage // target['hit_points'])
                target['units'] -= units_killed

        # Remove defeated groups
        immune_system = [g for g in immune_system if g['units'] > 0]
        infection = [g for g in infection if g['units'] > 0]

    return immune_system, infection

# Run the simulation
immune_system, infection = parse_input(data)
immune_system, infection = simulate_battle(immune_system, infection)
immune_system, infection

([],
 [{'units': 8004,
   'hit_points': 11443,
   'attack_damage': 2,
   'attack_type': 'cold',
   'initiative': 14,
   'weaknesses': [],
   'immunities': ['bludgeoning', 'slashing', 'fire']},
  {'units': 4426,
   'hit_points': 36617,
   'attack_damage': 16,
   'attack_type': 'cold',
   'initiative': 1,
   'weaknesses': ['radiation'],
   'immunities': []},
  {'units': 197,
   'hit_points': 18782,
   'attack_damage': 98,
   'attack_type': 'slashing',
   'initiative': 10,
   'weaknesses': ['radiation'],
   'immunities': ['fire']},
  {'units': 411,
   'hit_points': 17737,
   'attack_damage': 25,
   'attack_type': 'slashing',
   'initiative': 18,
   'weaknesses': ['slashing'],
   'immunities': ['bludgeoning', 'fire']},
  {'units': 99,
   'hit_points': 19361,
   'attack_damage': 104,
   'attack_type': 'fire',
   'initiative': 9,
   'weaknesses': ['bludgeoning', 'slashing'],
   'immunities': []},
  {'units': 2328,
   'hit_points': 30650,
   'attack_damage': 16,
   'attack_type': 'slashing',


In [120]:
sum([x['units'] for x in infection])

19381

In [125]:
from tqdm.notebook import tqdm

for boost in tqdm(range(34,10000)):
  immune_system, infection = parse_input(data)
  for x in immune_system:
    x['attack_damage'] += boost
  immune_system, infection = simulate_battle(immune_system, infection)
  if len(infection) == 0: break
immune_system

  0%|          | 0/9966 [00:00<?, ?it/s]

[{'units': 129,
  'hit_points': 11516,
  'attack_damage': 140,
  'attack_type': 'cold',
  'initiative': 7,
  'weaknesses': [],
  'immunities': []},
 {'units': 363,
  'hit_points': 2273,
  'attack_damage': 38,
  'attack_type': 'radiation',
  'initiative': 17,
  'weaknesses': [],
  'immunities': []},
 {'units': 300,
  'hit_points': 2540,
  'attack_damage': 46,
  'attack_type': 'radiation',
  'initiative': 4,
  'weaknesses': [],
  'immunities': ['fire']},
 {'units': 355,
  'hit_points': 3705,
  'attack_damage': 39,
  'attack_type': 'fire',
  'initiative': 16,
  'weaknesses': [],
  'immunities': []},
 {'units': 231,
  'hit_points': 6698,
  'attack_damage': 60,
  'attack_type': 'slashing',
  'initiative': 15,
  'weaknesses': [],
  'immunities': ['radiation']},
 {'units': 1667,
  'hit_points': 9656,
  'attack_damage': 51,
  'attack_type': 'fire',
  'initiative': 6,
  'weaknesses': [],
  'immunities': ['radiation']}]

In [126]:
sum([x['units'] for x in immune_system])

3045